In [4]:
#please install scipy and pandas by:
# %pip install scipy
# %pip install pandas
import numpy as np
from scipy.io import wavfile
from scipy.signal import stft
import pandas as pd

In [3]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Functions
def wav2fft(Path):
    # Load an audio file
    file_path = Path  # Replace with the path to your audio file
    sr, y = wavfile.read(file_path)

    # Compute the STFT
    n_fft = 2048  # Number of FFT points (window size)
    hop_length = 512  # Hop length between frames
    f, t, Zxx = stft(y, fs=sr, nperseg=n_fft, noverlap=hop_length)

    # Convert amplitude spectrogram to dB scale
    Zxx_db = 10 * np.log10(np.abs(Zxx)+0.0001)
    return Zxx_db

def data2fft(y):
    # Compute the STFT
    n_fft = 2048  # Number of FFT points (window size)
    hop_length = 512  # Hop length between frames
    f, t, Zxx = stft(y, fs=8000, nperseg=n_fft, noverlap=hop_length)

    # Convert amplitude spectrogram to dB scale
    Zxx_db = 10 * np.log10(np.abs(Zxx)+0.0001)
    return Zxx_db


##
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost_function_reg(X, y, weights, lambda_reg):
    m = X.shape[0]
    h = sigmoid(X @ weights)
    epsilon = 1e-5
    reg_term = (lambda_reg / (2 * m)) * np.sum(np.square(weights[1:]))
    cost = -1/m * np.sum(y * np.log(h + epsilon) + (1 - y) * np.log(1 - h + epsilon)) + reg_term
    return cost

def gradient_descent_reg(X, y, weights, learning_rate, iterations, lambda_reg):
    m = X.shape[0]
    cost_history = []

    for _ in range(iterations):
        weights[0] -= (learning_rate / m) * np.sum(sigmoid(X @ weights) - y)
        weights[1:] -= (learning_rate / m) * (X[:, 1:].T @ (sigmoid(X @ weights) - y) + lambda_reg * weights[1:])
        cost_history.append(cost_function_reg(X, y, weights, lambda_reg))

    return weights, cost_history

def train_one_vs_rest(X, y, num_classes, learning_rate, iterations, lambda_reg):
    models = []
    m, n = X.shape
    X = np.hstack((np.ones((m, 1)), X))

    for i in range(num_classes):
        y_ovr = np.where(y == i, 1, 0)
        weights = np.zeros(n + 1)
        weights, _ = gradient_descent_reg(X, y_ovr, weights, learning_rate, iterations, lambda_reg)
        models.append(weights)

    return models

def predict(X, models):
    m = X.shape[0]
    X = np.hstack((np.ones((m, 1)), X))
    preds = np.array([sigmoid(X @ w) for w in models]).T
    return np.argmax(preds, axis=1)

# Manual Data Splitting Function
def split_data(data, train_frac, test_frac):
    train_size = int(len(data) * train_frac)
    test_size = int(len(data) * test_frac)
    train_data = data[:train_size]
    validation_data = data[train_size:train_size + test_size]
    test_data = data[train_size + test_size:]
    return train_data, validation_data, test_data

# Manual Feature Scaling Function
def standardize_data(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / std

In [6]:
# Train and test classifier
v1=wav2fft("./ENGR_518_group_14_datasets/wav1.wav")[0:150,1:3000].transpose()
v2=wav2fft("./ENGR_518_group_14_datasets/wav2.wav")[0:150,1:3000].transpose()
v3=wav2fft("./ENGR_518_group_14_datasets/wav3.wav")[0:150,1:3000].transpose()


data = []
df = pd.DataFrame.from_records(v1)
df['label'] = 0
data.append(df)
df = pd.DataFrame.from_records(v2)
df['label'] = 1
data.append(df)
df = pd.DataFrame.from_records(v3)
df['label'] = 2
data.append(df)

data = pd.concat(data, ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)

# Splitting Data
train_data, validation_data, test_data = split_data(data, 0.7, 0.15)
X_train, y_train = train_data.drop('label', axis=1), train_data['label']
X_val, y_val = validation_data.drop('label', axis=1), validation_data['label']
X_test, y_test = test_data.drop('label', axis=1), test_data['label']

# Feature Scaling
#X_train_scaled= standardize_data(X_train)
#X_val_scaled= standardize_data(X_val)
#X_test_scaled= standardize_data(X_test)
X_train_scaled= (X_train)
X_val_scaled= (X_val)
X_test_scaled= (X_test)

# Training Parameters
learning_rate = 0.01
iterations = 3000
num_classes = 3
lambda_reg = 0.1

# Training the Models
models_reg = train_one_vs_rest(X_train_scaled, y_train, num_classes, learning_rate, iterations, lambda_reg)

np.save("subject1.weight150",models_reg[0])
np.save("subject2.weight150",models_reg[1])
np.save("subject3.weight150",models_reg[2])

# Validation and Testing
y_val_pred_reg = predict(X_val_scaled, models_reg)
accuracy_val = np.mean(y_val_pred_reg == y_val)

y_test_pred = predict(X_test_scaled, models_reg)
accuracy_test = np.mean(y_test_pred == y_test)

# Print the accuracy results
print(f"Validation Accuracy: {accuracy_val}")
print(f"Test Accuracy: {accuracy_test}")

Validation Accuracy: 0.9458858413639734
Test Accuracy: 0.9459659511472983


In [7]:
## Boosting, this loops will take a long time
# go get a cup of coffee while it is running, or maybe two

lambda_reg = 2**18
ind=1
while lambda_reg>0.001:


    # Training the Models
    for repeats in range(0,10):
        data = data.sample(frac=1).reset_index(drop=True)

        # Splitting Data
        train_data, validation_data, test_data = split_data(data, 0.7, 0.15)
        X_train, y_train = train_data.drop('label', axis=1), train_data['label']
        X_val, y_val = validation_data.drop('label', axis=1), validation_data['label']
        X_test, y_test = test_data.drop('label', axis=1), test_data['label']

        X_train_scaled = (X_train)
        X_val_scaled = (X_val)
        X_test_scaled = (X_test)

        models_reg = train_one_vs_rest(X_train_scaled, y_train, num_classes, learning_rate, iterations, lambda_reg)

        # Validation and Testing
        y_val_pred_reg = predict(X_val_scaled, models_reg)
        accuracy_val = np.mean(y_val_pred_reg == y_val)

        y_test_pred = predict(X_test_scaled, models_reg)
        accuracy_test = np.mean(y_test_pred == y_test)

        # Print the accuracy results
        print(f"Lamba:\t{lambda_reg}\tValidation Accuracy:\t{accuracy_val}\tTest Accuracy:\t{accuracy_test}")

        pd.DataFrame.from_records(models_reg).to_csv("./ENGR_518_group_14_datasets/boosting/boost_%03d_%03d.csv"%(ind,repeats), header=False, index=False)
    lambda_reg/=2
    ind+=1


Lamba:	262144	Validation Accuracy:	0.31653076352853965	Test Accuracy:	0.3168023686158401
Lamba:	262144	Validation Accuracy:	0.4618235730170497	Test Accuracy:	0.47964470762398226
Lamba:	262144	Validation Accuracy:	0.3313565604151223	Test Accuracy:	0.3449296817172465
Lamba:	262144	Validation Accuracy:	0.33209785025945143	Test Accuracy:	0.35899333826794966
Lamba:	262144	Validation Accuracy:	0.3157894736842105	Test Accuracy:	0.33530717986676534
Lamba:	262144	Validation Accuracy:	0.30911786508524836	Test Accuracy:	0.34863064396743154
Lamba:	262144	Validation Accuracy:	0.33432171979243885	Test Accuracy:	0.35899333826794966
Lamba:	262144	Validation Accuracy:	0.3617494440326168	Test Accuracy:	0.3441894892672095
Lamba:	262144	Validation Accuracy:	0.32765011119347665	Test Accuracy:	0.33974833456698744
Lamba:	262144	Validation Accuracy:	0.34840622683469236	Test Accuracy:	0.33530717986676534
Lamba:	131072.0	Validation Accuracy:	0.4032616753150482	Test Accuracy:	0.42116950407105846
Lamba:	131072.0	

In [8]:
selectFreq=[2,5,7,8,12,15]
# Load Data
v1=wav2fft("./ENGR_518_group_14_datasets/wav1.wav")[selectFreq,1:3000].transpose()
v2=wav2fft("./ENGR_518_group_14_datasets/wav2.wav")[selectFreq,1:3000].transpose()
v3=wav2fft("./ENGR_518_group_14_datasets/wav3.wav")[selectFreq,1:3000].transpose()

data = []
df = pd.DataFrame.from_records(v1)
df['label'] = 0
data.append(df)
df = pd.DataFrame.from_records(v2)
df['label'] = 1
data.append(df)
df = pd.DataFrame.from_records(v3)
df['label'] = 2
data.append(df)

data = pd.concat(data, ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)

# Splitting Data
train_data, validation_data, test_data = split_data(data, 0.7, 0.15)
X_train, y_train = train_data.drop('label', axis=1), train_data['label']
X_val, y_val = validation_data.drop('label', axis=1), validation_data['label']
X_test, y_test = test_data.drop('label', axis=1), test_data['label']

# Feature Scaling
#X_train_scaled= standardize_data(X_train)
#X_val_scaled= standardize_data(X_val)
#X_test_scaled= standardize_data(X_test)
X_train_scaled= (X_train)
X_val_scaled= (X_val)
X_test_scaled= (X_test)

# Training Parameters
learning_rate = 0.01
iterations = 3000
num_classes = 3
lambda_reg = 0.1

# Training the Models
models_reg = train_one_vs_rest(X_train_scaled, y_train, num_classes, learning_rate, iterations, lambda_reg)

# Validation and Testing
y_val_pred_reg = predict(X_val_scaled, models_reg)
accuracy_val = np.mean(y_val_pred_reg == y_val)

y_test_pred = predict(X_test_scaled, models_reg)
accuracy_test = np.mean(y_test_pred == y_test)

# Print the accuracy results
print(f"Validation Accuracy: {accuracy_val}")
print(f"Test Accuracy: {accuracy_test}")


Validation Accuracy: 0.8176426982950333
Test Accuracy: 0.8105107327905255
